In [ ]:
%pip install fredapi
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install scipy
%pip install statsmodels
%pip install pandas_market_calendars
%pip install tensorflow
%pip install keras

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.r

In [ ]:
dbutils.library.restartPython()

In [ ]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 
import scipy.interpolate as spi
from scipy.stats import hmean, mstats
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf
import datetime
import pandas_market_calendars as mcal
from fredapi import Fred
from keras.models import Sequential, load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam
import optree

"""
Setup
    user input
    configure date range 
    retrieve data 
    transform - dynamic nan handling 
    transform - enforce types
"""    



#USER INPUT PLUG WILL OBVIOUSLY BE ALTERED 
def userinput_point():

    # obviously we will be adding more - smaller for simplicity
    commod_map = {"WTI":'WTISPLC',"NATGAS":'MHHNGSP','WHEAT':'PWHEAMTUSDM','CORN':'PMAIZMTUSDM','SOYBEANS':'PSOYBUSDM'} 
    market_proxy = 'PALLFNFINDEXM'
    commod_map_vals = commod_map.values()
    commod_map_keys = commod_map.keys()
    #convert to lists
    commod_map_vals = list(commod_map.values())
    commod_map_keys = list(commod_map.keys())

    print(commod_map_vals)
    print(commod_map_keys)
    
    
    return commod_map, market_proxy, commod_map_vals, commod_map_keys



#ROLLING 20 - configure date range 
def date_config():

    #raw dates - 365
    todays_date = datetime.datetime.today()
    date_five_years_ago = todays_date - datetime.timedelta(365*20)

    #adjusted - define xchange dates via market proxy; S&P 500 PROXY NYSE- - 252
    exchange = 'XNYS'
    exchange_dates = mcal.get_calendar(exchange)
    trade_Range_dates = exchange_dates.schedule(start_date=date_five_years_ago, end_date=todays_date)

    closest_to_today = trade_Range_dates.index[-1].date()
    closest_To_date_fiveyearsago = trade_Range_dates.index[0].date()

    return closest_to_today, closest_To_date_fiveyearsago



#RETRIEVE DATA - user specified commodity and Market proxy = SPY or S&P 500 for simpleton
def data_retrieve(fred_api_key,commod_map_vals, commod_map_keys, market_proxy, closest_to_today, closest_To_date_fiveyearsago):

    #create empty list to quasi append to df transformation
    hist_df_list = []

    #get data from fred api for commodities
    for i in range(len(commod_map_keys)):
        fred_retrieve = fred_api_key.get_series(commod_map_vals[i],closest_To_date_fiveyearsago,closest_to_today)
        temp_df = pd.DataFrame(fred_retrieve, columns=[commod_map_keys[i]])
        hist_df_list.append(temp_df)
    
    fred_retrieve_df = pd.concat(hist_df_list, axis=1)
    
    #get market proxy benchmark 
    fred_retrieve_market = fred_api_key.get_series(market_proxy,closest_To_date_fiveyearsago,closest_to_today, frequency='m')
    fred_retrieve_market_df = pd.DataFrame(fred_retrieve_market, columns=['GLOBCOMINDX'])
    
    #concat both
    fred_retrieved_commod_market_df_concat = fred_retrieve_df.join(fred_retrieve_market_df)

    return fred_retrieve_df, fred_retrieve_market_df, fred_retrieved_commod_market_df_concat


    
#TRANSFORM DATA - NaNs
def transform_data_nans(fred_retrieved_commod_market_df_concat):
    
    row_count = len(fred_retrieved_commod_market_df_concat)
    
    for c in fred_retrieved_commod_market_df_concat.columns:
        #nans are random - gather metrics
        nans_count = fred_retrieved_commod_market_df_concat[c].isna().sum()
        print(f"{c} - NaNs: {nans_count}")
        
        threshold_2 = np.floor(0.02 * row_count)  #threshold - low echelon
        threshold_10 = np.floor(0.10 * row_count)  #threshold - middle echelon
        
        #drop if less than 2%
        if nans_count <= threshold_2:
            print('Drop used during transformation')
            fred_retrieved_commod_market_df_concat.dropna(subset=[c], inplace=True)
        
        # linear interpolate if between 2% and 10%
        elif nans_count <= threshold_10:
            print('Linear used during transformation')
            fred_retrieved_commod_market_df_concat[c] = fred_retrieved_commod_market_df_concat[c].interpolate(method='linear')
        
        #else use splines
        else:
            print('Spline used during transformation')
            non_nan_data = fred_retrieved_commod_market_df_concat[c].dropna()
            x = (non_nan_data.index - non_nan_data.index[0]).days  #cnvert index to numerical values
            y = non_nan_data.values

            #perform spline fitting - use cubic splines
            spline = spi.splrep(x, y, k=3)  

            #create x values for the entire range
            x_full = (fred_retrieved_commod_market_df_concat.index - fred_retrieved_commod_market_df_concat.index[0]).days

            #nterpolate missing values
            y_full = spi.splev(x_full, spline)

            #fill missing values in the original DataFrame
            fred_retrieved_commod_market_df_concat[c] = y_full

    transformed_df = fred_retrieved_commod_market_df_concat
    return transformed_df


#TRANSFORM DATA - enforce data types
def transform_data_enftypes(transformed_df):

    #enforce floats
    transformed_df = transformed_df.astype(float) 

    #enforce datetime index 
    transformed_df.index = pd.to_datetime(transformed_df.index)

    #title change
    final_df = transformed_df

    return final_df



"""
Initialize MOM Gradient
"""


def MOM_returns(final_df):

    #no log normal despite financial asset default right skew as we have not done any "model" yet

    for c in final_df.columns:
        final_df[f'{c} MOM_Nominal_Returns_decim.'] = (final_df[c] - final_df[c].shift(1)) / final_df[c].shift(1)

    #drop imputation error if occures
    final_df = final_df.dropna() 

    return final_df 



"""
# Computations

#     cummulative and incremental returns 
#         mean, harmonics, user specified time frames 
#     correlations to market 
#     sticky assets - autocorr via lag series
#     market betas via slope method
#     simple alpha identification
#     sharpe ratio
#     CAPM expected returns 
#     implied vol 
#     SMA and EMA
# """




#geom cumm returns 
def geometric_cummulative_return_monthly(final_df):

    #create a copy of the final transformed df
    final_df_geom = final_df.copy()

    #identify columns containing 'MOM_Nominal_Returns_decim.' to find monthly return columns
    mom_columns = [c for c in final_df_geom.columns if 'MOM_Nominal_Returns_decim.' in c]

    #set cumulative MOM geometric cumulative return for each MOM return column
    for c in mom_columns:
        #calculate cumulative geometric return
        final_df_geom[f'{c} Cumulative_Geometric_Return_MOM'] = (1 + final_df_geom[c]).cumprod() - 1

    #drop columns that are not geometric 
    non_geom_return_columns = [c for c in final_df_geom.columns if 'Cumulative_Geometric_Return_MOM' not in c]
    final_df_geom = final_df_geom.drop(columns=non_geom_return_columns)
    final_df_geom.columns = [f'{col.split(" ")[0]} Cumulative_Geometric_Return_MOM' for col in final_df_geom.columns]

    ###Note### - we could do this in quicksight
    #define cummulative MOM geom return at end of period -if you hypotheticallt held it for 20 years
    #decimal % we could make if held for 20 years... if 0.96 we make 96% return so youd double your investment

    return final_df_geom




"""
# Computations

#     cummulative and incremental returns 
#         mean, harmonics, user specified time frames 
#     correlations to market 
#     sticky assets - autocorr via lag series
#     market betas via slope method
#     simple alpha identification
#     sharpe ratio
#     CAPM expected returns 
#     implied vol 
#     SMA and EMA
# """




#geom cumm returns 
def geometric_cummulative_return_monthly(final_df):

    #create a copy of the final transformed df
    final_df_geom = final_df.copy()

    #identify columns containing 'MOM_Nominal_Returns_decim.' to find monthly return columns
    mom_columns = [c for c in final_df_geom.columns if 'MOM_Nominal_Returns_decim.' in c]

    #set cumulative MOM geometric cumulative return for each MOM return column
    for c in mom_columns:
        #calculate cumulative geometric return
        final_df_geom[f'{c} Cumulative_Geometric_Return_MOM'] = (1 + final_df_geom[c]).cumprod() - 1

    #drop columns that are not geometric 
    non_geom_return_columns = [c for c in final_df_geom.columns if 'Cumulative_Geometric_Return_MOM' not in c]
    final_df_geom = final_df_geom.drop(columns=non_geom_return_columns)
    final_df_geom.columns = [f'{col.split(" ")[0]} Cumulative_Geometric_Return_MOM' for col in final_df_geom.columns]

    ###Note### - we could do this in quicksight
    #define cummulative MOM geom return at end of period -if you hypotheticallt held it for 20 years
    #decimal % we could make if held for 20 years... if 0.96 we make 96% return so youd double your investment

    return final_df_geom

# annual geom returns
def annual_geometric_returns(final_df):

    #create copy of final transformed df
    final_df_annual_geom = final_df.copy()

    #mom colums 
    mom_columns_geoannual = [c for c in final_df_annual_geom.columns if 'MOM_Nominal_Returns_decim.' in c]
   
    #init dict
    annual_returns_dict = {
        year: {
            c: round((1 + final_df_annual_geom.loc[final_df_annual_geom.index.year == year, c]).prod() - 1, 2)
            for c in mom_columns_geoannual
            }
            for year in final_df_annual_geom.index.year.unique()
        }

    final_df_annual_geom = pd.DataFrame.from_dict(annual_returns_dict, orient='index')
    
    #rename columns
    final_df_annual_geom.columns = [f'{col.split(" ")[0]} Annual_Geometric_Return_MOM' for col in final_df_annual_geom.columns]

    return final_df_annual_geom


#arithmatic average monthly return over entire period fro all months
def arithmatic_avg_return_monthly(final_df):

    #create copy of final transformed df
    final_df_ang_monthly_arith = final_df.copy()

    #mom colums 
    mom_columns_arithavg = [c for c in final_df_ang_monthly_arith.columns if 'MOM_Nominal_Returns_decim.' in c]

    monthly_arith_return_averages = {
        c: round(np.mean(final_df_ang_monthly_arith[c]), 4) for c in mom_columns_arithavg
    }
    
    #convert to df
    monthly_arith_return_averages_df = pd.DataFrame([monthly_arith_return_averages], index=['Average_Monthly_Arithmetic_Return'])
    #rename col
    monthly_arith_return_averages_df.columns = [f'{col.split(" ")[0]} monthly_Arith_avgReturn_MOM' for col in monthly_arith_return_averages_df.columns]
    monthly_arith_return_averages_df.columns = [col.replace('monthly_Arith_avgReturn_MOM', '') for col in monthly_arith_return_averages_df.columns]

    return monthly_arith_return_averages_df

#arithmatic average for each year over entire period 
def arithmatic_avg_return_annual(final_df):

    #create copy of final transformed df
    final_df_avg_annual_arith = final_df.copy()
    #define columns 
    mom_columns_geoannual = [c for c in final_df_avg_annual_arith.columns if 'MOM_Nominal_Returns_decim.' in c]

    annual_arithavg_returns = {
        year: {
            c: round(final_df_avg_annual_arith.loc[final_df_avg_annual_arith.index.year == year, c].mean(), 2)
            for c in mom_columns_geoannual
        }
        for year in final_df_avg_annual_arith.index.year.unique()
    }

    #to dataframe 
    final_df_avg_annual_arith = pd.DataFrame.from_dict(annual_arithavg_returns, orient='index')
    #change col names
    final_df_avg_annual_arith.columns = [f'{col.split(" ")[0]} Arith_avgAnnualReturn' for col in final_df_avg_annual_arith.columns]

    return final_df_avg_annual_arith


#windsored average monthly return over entire period fro all months
def windsored_avg_return_monthly(final_df):

    #create copy of final transformed df
    final_df_ang_monthly_windsored = final_df.copy()

    #mom colums 
    mom_columns_winmonthly = [c for c in final_df_ang_monthly_windsored.columns if 'MOM_Nominal_Returns_decim.' in c]

    #proportion to windsor 
    proportion_to_winsorize_m = 0.05

    #windsor data - windsor data then take mean
    winsorized_means = {
        column: round(np.mean(mstats.winsorize(final_df_ang_monthly_windsored[column], limits=proportion_to_winsorize_m)), 4)
        for column in mom_columns_winmonthly
    }
    final_df_ang_monthly_windsored = pd.DataFrame([winsorized_means], index=['Windsored_Avg_Monthly_Return'])
    final_df_ang_monthly_windsored.columns = [col.replace('MOM_Nominal_Returns_decim.', '') for col in final_df_ang_monthly_windsored.columns]

    return final_df_ang_monthly_windsored

#windsored average for each year over entire period 
def windsored_avg_return_annual(final_df):

    #create copy of final transformed df
    final_df_ang_annual_windsored = final_df.copy()

    #mom def 
    mom_columns_winsannual = [c for c in final_df_ang_annual_windsored.columns if 'MOM_Nominal_Returns_decim.' in c]

    #proportion to windsor 
    proportion_to_winsorize_y = 0.02

    #windsor data
    windsored_annual_returns = {
    column: {
        year: round(final_df_ang_annual_windsored.loc[
            final_df_ang_annual_windsored.index.year == year,
            column + '_Winsorized'].mean(), 2)
        for year in final_df_ang_annual_windsored.index.year.unique()
    }
    for column in mom_columns_winsannual
    for final_df_ang_annual_windsored[column + '_Winsorized'] in [mstats.winsorize(final_df_ang_annual_windsored[column], limits=proportion_to_winsorize_y)]
    }

    final_df_ang_annual_windsored = pd.DataFrame(windsored_annual_returns)

    return final_df_ang_annual_windsored

#historical std dev sample monthly 
def std_dev_sample_monthly(final_df):

    #create copy of final transformed df
    final_df_historical_monthly_stddev_samp = final_df.copy()

    #mom drop
    mom_columns_winsannual = [c for c in final_df_historical_monthly_stddev_samp.columns if 'MOM_Nominal_Returns_decim.' in c]

    #std dev sample
    std_dev_samples = {
        column: round(final_df_historical_monthly_stddev_samp[column].std(), 2)
        for column in mom_columns_winsannual
    }

    final_df_historical_monthly_stddev_samp = pd.DataFrame([std_dev_samples], index=['Standard_Deviation_20Y'])

    #rename columns
    final_df_historical_monthly_stddev_samp.columns = [f'{col.split(" ")[0]} StdDev_Monthly' for col in final_df_historical_monthly_stddev_samp.columns]
    final_df_historical_monthly_stddev_samp.columns = [col.replace('StdDev_Monthly', '') for col in final_df_historical_monthly_stddev_samp.columns]

    return final_df_historical_monthly_stddev_samp

#historical std dev sample per year
def std_dev_sample_annual(final_df):

    #create copy of final transformed df
    final_df_historical_annual_stddev_samp = final_df.copy()

    #mom drop
    mom_columns_stddevannual = [c for c in final_df_historical_annual_stddev_samp.columns if 'MOM_Nominal_Returns_decim.' in c]

    #annual std dev samples 
    std_dev_sample_annual_dict = {
    column: {
        year: round(final_df_historical_annual_stddev_samp.loc[final_df_historical_annual_stddev_samp.index.year == year, column].std(), 2)
        for year in final_df_historical_annual_stddev_samp.index.year.unique()
        }
    for column in mom_columns_stddevannual
    }
    
    std_dev_sample_annual_df = pd.DataFrame(std_dev_sample_annual_dict)

    #rename columns 
    std_dev_sample_annual_df.columns = [f'{col.split(" ")[0]} StdDev_Annual' for col in std_dev_sample_annual_df.columns]


    return std_dev_sample_annual_df

#minimum month and corresponding date
def min_return_month(final_df):

    #create copy of final transformed df
    final_df_min = final_df.copy()

    #mom 
    min_cols = [c for c in final_df_min.columns if 'MOM_Nominal_Returns_decim.' in c]

    min_info = {
    column: {
        'Minimum Return': final_df_min[column].min(),
        'Corresponding Min Index': final_df_min[column].idxmin()
        } for column in min_cols
    }

    min_returns_df = pd.DataFrame(min_info).transpose()
    min_returns_df.reset_index(inplace=True)
    min_returns_df.rename(columns={'index': 'Column'}, inplace=True)

    return min_returns_df

#maximum month and corresponding date
def max_return_month(final_df):

    #create copy of final transformed df
    final_df_max = final_df.copy()

    #mom
    max_cols = [c for c in final_df_max.columns if 'MOM_Nominal_Returns_decim.' in c]
    
    #get maximum and corresponding maximum index
    max_info = {
        column: {
            'Minimum Return': final_df_max[column].max(),
            'Corresponding Min Index': final_df_max[column].idxmax()
        } for column in max_cols
    }

    max_returns_df = pd.DataFrame(max_info).transpose()
    max_returns_df.reset_index(inplace=True)
    max_returns_df.rename(columns={'index': 'Column'}, inplace=True)

    return max_returns_df

#sma
def sma_monthly(final_df):

    #create copy of final transformed df
    final_df_sma = final_df.copy()

    #mom
    sma_cols = [c for c in final_df_sma.columns if 'MOM_Nominal_Returns_decim.' in c]

    #sma calc - rolling window
    for column in sma_cols:
        final_df_sma[f'Rolling5MSMA_{column}'] = final_df_sma[column].rolling(window=5).mean()

    #eliminate imputation nans 
    final_df_sma = final_df_sma.dropna(how='all', subset=[f'Rolling5MSMA_{col}' for col in sma_cols])

    #drop columns
    final_df_sma = final_df_sma.filter(regex='Rolling5MSMA')
    #rename columns 
    final_df_sma.columns = [f"{col.split('_', 1)[1]} Rolling5mSMA" for col in final_df_sma.columns]
    final_df_sma.columns = [f"{col.split(' MOM_Nominal_Returns_decim.')[0]} Rolling5mSMA" for col in final_df_sma.columns]

    return final_df_sma 

#ema
def ema_monthly(final_df):

    #create copy of final transformed df
    final_df_ema = final_df.copy()

    #mom
    ema_cols = [c for c in final_df_ema.columns if 'MOM_Nominal_Returns_decim.' in c]

    #ema calc = rolling window
    for column in ema_cols:
        final_df_ema[f'Rolling5MEMA_{column}'] = final_df_ema[column].ewm(span=5, adjust=False).mean()

    #emininate nans
    final_df_ema = final_df_ema.dropna(how='all', subset=[f'Rolling5MEMA_{col}' for col in ema_cols])

     #drop columns
    final_df_ema = final_df_ema.filter(regex='Rolling5MEMA')
    #rename columns 
    final_df_ema.columns = [f"{col.split('_', 1)[1]} Rolling5mEMA" for col in final_df_ema.columns]
    final_df_ema.columns = [f"{col.split(' MOM_Nominal_Returns_decim.')[0]} Rolling5mEMA" for col in final_df_ema.columns]

    return final_df_ema 


#autocrrelation 
# could help with identifying seasinal effect, or risk managemnte stechniques (if highly positively or negatively correlated that infleunces buy, hold sell)
def autocorr_dectect(final_df):
    #create copy of final transformed df
    final_df_autcorr = final_df.copy()
    #indentify columns
    acf_cols = [c for c in final_df_autcorr.columns if 'MOM_Nominal_Returns_decim.' in c]
    #values
    acf_df = pd.DataFrame()
    for column in acf_cols:
        acf_values = acf(final_df_autcorr[column], nlags=30)
        acf_df[column] = acf_values
    #set index to represent lags
    acf_df.index = range(len(acf_values))
    acf_df.index.name = 'Lag'
    #rename columns
    acf_df.columns = [f"{col.split('_', 1)[0]} AutoCorr_Lag" for col in acf_df.columns]
    

    # plot_pacf(final_df_autcorr['MOM_Nominal_Returns_decim.'], lags=30)
    # plt.xlabel('N Days Lag (Days Back)')
    # plt.ylabel('Corr Coeff')
    # plt.show()

    return acf_df

#corr to market
def corr_to_market(final_df):

    #make copy 
    final_df_corr = final_df.corr()
    #get only price sof commodities
    half_columns_df = final_df.iloc[:, :len(final_df.columns) // 2]
    #get matrix
    prices_df = half_columns_df.corr()


    return prices_df

#beta via slope method - covar(s0,m0) / var(m0)
def beta_twentyY(final_df):

    #make copy 
    final_df_beta = final_df.copy()
    
    #specify columns
    beta_cols = [c for c in final_df_beta.columns if 'MOM_Nominal_Returns_decim.' in c]
    
    beta_values = {
    c: np.cov(final_df_beta[c], final_df_beta.iloc[:,-1])[0, 1] / np.var(final_df_beta.iloc[:,-1])
        for c in beta_cols
    }

    final_df_beta = pd.DataFrame(beta_values, index=['Beta_Values_20Y'])
    final_df_beta.columns = [col.replace('MOM_Nominal_Returns_decim.', '') for col in final_df_beta.columns]

    return final_df_beta

#beta for 5 years
def beta_fiveY(final_df):
    
    #make copy 
    final_df_beta_five = final_df.copy()
    
    #filter last 5 years
    last_5_years = final_df_beta_five.index > (final_df_beta_five.index.max() - pd.DateOffset(years=5))
    filtered_df = final_df_beta_five.loc[last_5_years]
    #specify columns
    beta_cols_f = [c for c in filtered_df.columns if 'MOM_Nominal_Returns_decim.' in c]
    
    beta_values_f = {
    c: np.cov(filtered_df[c], filtered_df.iloc[:,-1])[0, 1] / np.var(filtered_df.iloc[:,-1])
        for c in beta_cols_f
    }

    final_df_beta_five = pd.DataFrame(beta_values_f, index=['Beta_Values_5Y'])
    final_df_beta_five.columns = [col.replace('MOM_Nominal_Returns_decim.', '') for col in final_df_beta_five.columns]
    #print(final_df_beta_five)

    return final_df_beta_five

#sharpe ratio - return - rf / std dev
def sharpe_ratio_tewentyY(final_df, annual_arithavg_returns,std_dev_sample_annual_df,rf_oneY_avg=0.0451):

    #exclude rf rate as we are dealing with commodites opposed to traditonal
    #adjsut returns 
    averages_annual_sharpe_return =  annual_arithavg_returns.mean().to_frame().T
    averages_annual_sharpe_return.columns = [col.replace('MOM_Nominal_Returns_decim', 'Arith_avgAnnualReturn') for col in averages_annual_sharpe_return.columns]
    #adjsut std deviation
    avg_annual_stddev_sharpe =  std_dev_sample_annual_df.mean().to_frame().T
    avg_annual_stddev_sharpe.columns = [col.replace('MOM_Nominal_Returns_decim', 'Stddev_AnnualAvg') for col in avg_annual_stddev_sharpe.columns]
    #sharpe calc
    sharpe_ratio_annual_avg_final_df = averages_annual_sharpe_return.div(avg_annual_stddev_sharpe.values)
    sharpe_ratio_annual_avg_final_df.columns = [col.replace('_Arith_avgAnnualReturn','') for col in averages_annual_sharpe_return.columns]
    sharpe_ratio_annual_avg_final_df = sharpe_ratio_annual_avg_final_df.rename(index = {0:'Avg_Annual_SharpeRatio'})
    
    return sharpe_ratio_annual_avg_final_df

def capm_er(final_df,final_df_beta,final_df_avg_annual_arith,rf_oneY_avg_beta=0.0451):

    #average annual return of market 
    avg_annual_return_of_the_market = np.mean(final_df_avg_annual_arith.iloc[:,-1])

    #alter_beta not requried as vol meansure doesnt typically require componuntign for time 
    capm_er_df = final_df_beta * (avg_annual_return_of_the_market - rf_oneY_avg_beta) + rf_oneY_avg_beta
    capm_er_df.columns = [col.replace('20YBeta.', '') for col in capm_er_df.columns]
    capm_er_df = capm_er_df.rename(index = {'Beta_Values_20Y':'CAPM_Er_Avg_Annual'})
    
    return capm_er_df



"""
Line table concatination - FORMATTING
"""
def line_table_concat(monthly_arith_return_average,winsorized_mean_monthly,
                      std_dev_sample,final_df_beta,final_df_beta_five,sharpe_ratio_annual_avg_final_df,capm_er_df):
    
    #transform lines 
    columns = final_df_beta.columns
    df_list_table = [final_df_beta,final_df_beta_five,sharpe_ratio_annual_avg_final_df,
                             capm_er_df,monthly_arith_return_average,winsorized_mean_monthly,std_dev_sample]
    for df in df_list_table:
        df.columns = columns
    
    metric_table = pd.concat(df_list_table, axis=0)

    return metric_table





"""
Machine Learning Model
"""

#data plug all 
def data_plug(final_df):
    final_df = final_df.reset_index()
    print(final_df)
    price_df = final_df
    user_input = ['WTI MOM_Nominal_Returns_decim.',	'NATGAS MOM_Nominal_Returns_decim.','WHEAT MOM_Nominal_Returns_decim.','CORN MOM_Nominal_Returns_decim.','SOYBEANS MOM_Nominal_Returns_decim.','GLOBCOMINDX MOM_Nominal_Returns_decim.']
    

    price_df_final = price_df[['index'] + user_input].iloc[:,:-1]
    

    return price_df_final

#data clean all
def data_clean(price_df_final):
    
    #date index
    price_df_final['index'] = pd.to_datetime(price_df_final['index'])
    price_df_final['index'] = price_df_final['index'].dt.strftime('%Y-%m-%d')
    price_df_final = price_df_final.set_index('index',inplace = False)
    #numeric price
    price_df_final.iloc[:,0] = pd.to_numeric(price_df_final.iloc[:,0])

    #dropna()
    price_df_final = price_df_final.interpolate()
    print(price_df_final)
    return price_df_final

#helper input an output
def input_and_output(price_df_final):  
    


    n_inputs = 5 #due to shortterm sequential dependancies
    n_features = 1 #as we are dealing with one commodity

    hist_df_2d = price_df_final.iloc[:,[0]] # 2d
    hist_df_2d = hist_df_2d.to_numpy() #convert to nupy array
    
    #x input and y state lists
    x_Input_list = []
    y_State_list = []

    for i in range(len(hist_df_2d)-n_inputs): # maintain number of inputs for sequential dependancy for purpose of ensuring we do not go out of bounds
        row_Batch = [[a] for a in hist_df_2d[i:i+n_inputs]] #batches construct
        x_Input_list.append(row_Batch)

        label = hist_df_2d[i+2]
        y_State_list.append(label)

    #convert back to array to develop and allow for formation of shape
    x_Input_list_array = np.array(x_Input_list)
    y_Input_list_array = np.array(y_State_list)

    list_Point_X1 = len(x_Input_list_array) // 3
    list_Point_X2 = 2 * list_Point_X1#endpoint of validation

    list_Point_Y1 = len(y_Input_list_array) // 3
    list_Point_Y2 = 2 * list_Point_Y1#endpoint of validation

    #training - 1/3
    train_Data_x = np.array(x_Input_list[:list_Point_X1])
    train_Data_y = np.array(y_State_list[:list_Point_Y1])
    
    #validation - 2/3
    validation_Data_X = np.array(x_Input_list[list_Point_X1:list_Point_X2])
    validation_Data_y = np.array(y_State_list[list_Point_Y1:list_Point_Y2])

    #test - 3/3
    test_Data_x = np.array(x_Input_list[list_Point_X2:])
    test_Data_y = np.array(y_State_list[list_Point_Y2:])
    
    return (n_inputs, n_features, train_Data_x, train_Data_y, validation_Data_X, 
            validation_Data_y, test_Data_x, test_Data_y)


def lstm_model(n_inputs, n_features, train_Data_x, train_Data_y, validation_Data_X, 
            validation_Data_y, test_Data_x, test_Data_y):
    
    #define model
    lstm_model = Sequential() # sequential to simplify model construction 
    lstm_model.add(InputLayer((n_inputs, n_features)))
    lstm_model.add(LSTM(100)) # 100 nuerons decent for complex data
    lstm_model.add(Dense(8, 'relu')) # reltive linear unit activiation, good for handling complex data patterns
    lstm_model.add(Dense(1, 'linear')) # output layer | linear makes good for regression tasks where model expected to predict continuous outpuit
    print(lstm_model.summary())
    #callback
    cp = ModelCheckpoint('lstm_Model/best_model.keras', save_best_only=True) # save only best at continous points during training and save best 
    #compile and save best in lstm directory
    lstm_model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

    #run epoch
    epoch_run_through = lstm_model.fit(train_Data_x, train_Data_y,validation_data=(validation_Data_X,validation_Data_y), epochs=45, callbacks=[cp])
    train_loss = round(epoch_run_through.history['loss'][-1],3)
    train_root_mse = round(epoch_run_through.history['root_mean_squared_error'][-1],3)
    print(train_loss)
    print(train_root_mse)
   
    ###graph loss after each epoch to see opimal one, inflection at 20### conf efficeintcy and overfitting 
    # loss_per_epoch = lstm_model.history.history['loss']
    # plt.plot(range(len(loss_per_epoch)),loss_per_epoch)
    # plt.show()

    """
    Predict Test
    """

    #load model with lowest validation 
    lstm_Model = load_model('lstm_Model/best_model.keras')

    #compare predicted testing (x) to actual or y test
    test_Predictions = lstm_Model.predict(test_Data_x).flatten() # remove inner bracakets 

    """
    Predict - Forecast 
    """
    num_future_predictions_months = 6
    future_predictions = []
    last_five_data_points = test_Data_x[-1:]

    #rolling prediction 
    for i in range(num_future_predictions_months):
        next_pred_role = lstm_Model.predict(last_five_data_points).flatten()
        future_predictions.append(next_pred_role)
        next_pred_role = np.append(last_five_data_points,next_pred_role)
        next_pred_role = next_pred_role[1:]
        next_pred_role = next_pred_role.reshape(1,5,1,1)
        #replace last_five_data_points with next_pred_role to enable batch role
        last_five_data_points = next_pred_role


    
    return test_Predictions, future_predictions


def train_test_forecast_all_commodites(price_df_final):
    commodities = ['WTI MOM_Nominal_Returns_decim.', 'NATGAS MOM_Nominal_Returns_decim.',
               'WHEAT MOM_Nominal_Returns_decim.', 'CORN MOM_Nominal_Returns_decim.',
               'SOYBEANS MOM_Nominal_Returns_decim.']

    #dict store results - train test
    results = {}
    future_pred = {}

    for commodity in commodities:
        price_df_final_commodity = price_df_final[[commodity]].dropna()  #select rel and drop nans

        (n_inputs, n_features, train_Data_x, train_Data_y, validation_Data_X, 
        validation_Data_y, test_Data_x, test_Data_y) = input_and_output(price_df_final_commodity)

        test_Predictions, future_predictions = lstm_model(n_inputs, n_features, train_Data_x, train_Data_y, validation_Data_X, validation_Data_y, test_Data_x,test_Data_y)

        results[commodity] = {
            'train_pred': test_Predictions,
            'test_Data_y': test_Data_y
        }
    
        future_pred[commodity] = {
            'forecast_months': future_predictions
        }

    return results, future_pred



def flatten(results, future_pred):
    flattened_data = []
    flattened_pred = []
    for commodity, values in results.items():
        train_pred = values['train_pred'].flatten()
        test_data_y = values['test_Data_y'].flatten()

        #calculate rsquared adj
        ss_res = np.sum((test_data_y - train_pred) ** 2)
        ss_tot = np.sum((test_data_y - np.mean(test_data_y)) ** 2)
        r_squared = 1 - (ss_res / ss_tot)
        n = len(test_data_y)
        p = 1
        adjusted_r_squared = 1 - (((1 - r_squared) * (n - 1)) / (n - p - 1))

        #test rmse 
        sse = np.sum((test_data_y - train_pred) ** 2)
        mse = sse / len(test_data_y)
        rmse = np.sqrt(mse)

        for pred, actual in zip(train_pred, test_data_y):
            flattened_data.append({
                'commodity': commodity,
                'train_pred': pred,
                'test_Data_y': actual,
                'R-Squared; Adjusted - Test' : adjusted_r_squared,
                'Root Mean Square Error (RMSE) - Test': rmse

            })

    # Create a DataFrame - test and pred
    df_flatten_test = pd.DataFrame(flattened_data)




    # Flatten the future predictions
    future_flattened_data = []
    for commodity, future_values in future_pred.items():
        forecast_months = future_values['forecast_months']
        for i, value in enumerate(forecast_months):
            future_flattened_data.append({
                'commodity': commodity,
                'month': i + 1,
                'forecast': value[0]  # Extract the value from the array
            })

    # Create a DataFrame for future predictions
    df_future_pred = pd.DataFrame(future_flattened_data)


    return df_flatten_test, df_future_pred









"""
Wrapper
"""


def main():

    #def api key:
    fred_api_key = Fred(api_key='2d1a840a6130007c85c02042377c38af')

    #wrap - setup
    commod_map, market_proxy, commod_map_vals, commod_map_keys = userinput_point()    
    closest_to_today, closest_To_date_fiveyearsago = date_config()
    fred_retrieve_df, fred_retrieve_market_df, fred_retrieved_commod_market_df_concat = data_retrieve(fred_api_key,commod_map_vals, commod_map_keys, market_proxy, closest_to_today, closest_To_date_fiveyearsago)
    transformed_df = transform_data_nans(fred_retrieved_commod_market_df_concat)

    #wrap - initialize base gradient returns 
    final_df = transform_data_enftypes(transformed_df)
    
    #mom returns 
    final_df = MOM_returns(final_df)

    # #wrap - computations
    final_cumm_geo = geometric_cummulative_return_monthly(final_df) #table
    annual_returns = annual_geometric_returns(final_df) #table
    monthly_arith_return_average = arithmatic_avg_return_monthly(final_df) #line
    final_df_avg_annual_arith = arithmatic_avg_return_annual(final_df) #table
    winsorized_mean_monthly = windsored_avg_return_monthly(final_df) #line
    final_df_ang_annual_windsored = windsored_avg_return_annual(final_df) #table
    std_dev_sample = std_dev_sample_monthly(final_df) #line
    std_dev_sample_annual_df = std_dev_sample_annual(final_df) #table
    min_returns_df = min_return_month(final_df) #table alt 
    max_returns_df = max_return_month(final_df) # table alt
    final_df_sma = sma_monthly(final_df) #table
    final_df_ema = ema_monthly(final_df) #table
    acf_values_returns = autocorr_dectect(final_df) #table
    corr_martix = corr_to_market(final_df) #table alt
    final_df_beta = beta_twentyY(final_df) #line
    final_df_beta_five = beta_fiveY(final_df) #line
    sharpe_ratio_annual_avg_final_df = sharpe_ratio_tewentyY(final_df, final_df_avg_annual_arith,std_dev_sample_annual_df,rf_oneY_avg=0.0451) #line
    capm_er_df = capm_er(final_df,final_df_beta,final_df_avg_annual_arith,rf_oneY_avg_beta=0.0441) #line

    #machine leatrning model 

    #dataplug
    price_df_final = data_plug(final_df)
    #clean
    price_df_final = data_clean(price_df_final)
    #input output helper
    (n_inputs, n_features, train_Data_x, train_Data_y, validation_Data_X, 
            validation_Data_y, test_Data_x, test_Data_y) = input_and_output(price_df_final)
    #lstm model helper
    test_Predictions, future_predictions = lstm_model(n_inputs, n_features, train_Data_x, train_Data_y, validation_Data_X, 
            validation_Data_y, test_Data_x, test_Data_y)
    #input output each commodity
    results, future_pred = train_test_forecast_all_commodites(price_df_final)
    #flatten
    df_flatten_test, df_future_pred = flatten(results, future_pred)



    #combine line tables - FORMATTING 
    metric_table = line_table_concat(monthly_arith_return_average,winsorized_mean_monthly,
                      std_dev_sample,final_df_beta,final_df_beta_five,sharpe_ratio_annual_avg_final_df,capm_er_df) # table
    
    return final_df,final_cumm_geo, annual_returns, final_df_avg_annual_arith, final_df_ang_annual_windsored, std_dev_sample_annual_df, final_df_sma, final_df_ema, acf_values_returns, corr_martix, min_returns_df, max_returns_df, metric_table, df_flatten_test, df_future_pred


final_df, final_cumm_geo, annual_returns, final_df_avg_annual_arith, final_df_ang_annual_windsored, std_dev_sample_annual_df, final_df_sma, final_df_ema, acf_values_returns, corr_martix, min_returns_df, max_returns_df, metric_table, df_flatten_test, df_future_pred = main()






2024-08-01 03:08:00.414524: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-01 03:08:00.417471: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 03:08:00.423308: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 03:08:00.442466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 03:08:00.478155: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

['WTISPLC', 'MHHNGSP', 'PWHEAMTUSDM', 'PMAIZMTUSDM', 'PSOYBUSDM']
['WTI', 'NATGAS', 'WHEAT', 'CORN', 'SOYBEANS']
WTI - NaNs: 0
Drop used during transformation
NATGAS - NaNs: 0
Drop used during transformation
WHEAT - NaNs: 0
Drop used during transformation
CORN - NaNs: 0
Drop used during transformation
SOYBEANS - NaNs: 0
Drop used during transformation
GLOBCOMINDX - NaNs: 0
Drop used during transformation
         index  ...  GLOBCOMINDX MOM_Nominal_Returns_decim.
0   2004-09-01  ...                               -0.001950
1   2004-10-01  ...                                0.071944
2   2004-11-01  ...                               -0.030356
3   2004-12-01  ...                               -0.014892
4   2005-01-01  ...                                0.034602
..         ...  ...                                     ...
233 2024-02-01  ...                               -0.004344
234 2024-03-01  ...                                0.027953
235 2024-04-01  ...                                0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100)            │        40,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │           808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41,617 (162.57 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 100)            │        40,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 8)              │           808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41,617 (162.57 KB)

 Trainable params: 41,617 (162.57 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/45
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 810ms/step - loss: 0.0053 - root_mean_squared_error: 0.0724 - val_loss: 0.0026 - val_root_mean_squared_error: 0.0513
Epoch 2/45
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0037 - root_mean_squared_error: 0.0603 - val_loss: 0.0029 - val_root_mean_squared_error: 0.0541
Epoch 3/45
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0045 - root_mean_squared_error: 0.0670 - val_loss: 0.0029 - val_root_mean_squared_error: 0.0539
Epoch 4/45
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 654ms/step - loss: 0.0040 - root_mean_squared_error: 0.0629 - val_loss: 0.0026 - val_root_mean_squared_error: 0.0512
Epoch 5/45
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 500ms/step - loss: 0.0038 - root_mean_squared_error: 0.0614 - val_loss: 0.0024 - val_root_mean_squared_error: 0.0489
Epoch 6/45
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 575ms/step - loss: 0.0036 - root_mean_squared_error: 0.0598 - val_loss: 0.0023 - val_root_mean_squared_error: 0.0478
Epoch 7/45
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step - loss: 0.0

In [ ]:
# Compile to s3 bucket
import boto3
from datetime import datetime 
import os

import pytz

# Define the timezone as Canada's Mountain Standard Time
mst = pytz.timezone('Canada/Mountain')

# Get the current date in MST
current_date = datetime.now(mst).strftime("%Y-%m-%d")

# Create a DataFrame with the current date
date_df = pd.DataFrame({'upload_date': [current_date]})


df_list_s3 = [final_df,final_cumm_geo, annual_returns, final_df_avg_annual_arith, 
              final_df_ang_annual_windsored, std_dev_sample_annual_df, final_df_sma,final_df_ema, acf_values_returns, corr_martix, min_returns_df, max_returns_df, metric_table,df_flatten_test, df_future_pred, date_df]

df_names = ["final_df","final_cumm_geo", "annual_returns", "final_df_avg_annual_arith", 
            "final_df_ang_annual_windsored", "std_dev_sample_annual_df", "final_df_sma",
            "final_df_ema", "acf_values_returns", "corr_martix", "min_returns_df", 
            "max_returns_df", "metric_table", "df_flatten_test", "df_future_pred", "date_df"]

# This will be used in the for loop below
s3_client = boto3.client('s3')
s3_bucket_name = os.getenv("S3_BUCKET_NAME")

# Assuming df_list_s3 and df_names are in the same order
for df, name in zip(df_list_s3, df_names):
    # Realize index
    df.reset_index(inplace=True)
    # Convert Pandas DataFrame to Spark DataFrame
    data_spark = spark.createDataFrame(df)
    data_spark_single_partition = data_spark.coalesce(1)
    # Specify S3 path with unique identifier for each DataFrame
    s3_bucket_name = s3_bucket_name
    s3_bucket_path = f"s3a://{s3_bucket_name}/"
    s3_tmp_path_prefix = f"data-folder/tmp/{name}.csv"
    s3_final_path_prefix = f"data-folder/{name}.csv"
    # Write the DataFrame to S3
    data_spark_single_partition.write.mode("overwrite")\
    .option("header", "true")\
    .option("delimiter", ",")\
    .option("quote", "\"")\
    .option("charset", "UTF-8")\
    .csv(s3_bucket_path + s3_tmp_path_prefix)

    # Renaming the csv files to df_names
    response = s3_client.list_objects_v2(Bucket=s3_bucket_name, Prefix=f"{s3_tmp_path_prefix}")
    csv_file = None
    for obj in response.get('Contents', []):
        if obj['Key'].endswith(".csv"):
            csv_file = obj['Key']
            break
    if csv_file:
    # Define source and destination paths
        source_key = csv_file
        destination_key = s3_final_path_prefix
    
    # Copy the file to the new location
        copy_source = {'Bucket': s3_bucket_name, 'Key': source_key}
        s3_client.copy_object(CopySource=copy_source, Bucket=s3_bucket_name, Key=destination_key)
    
    # Delete the original file
    # Uncomment the code below if you want the temporary csv file to be deleted
    #s3_client.delete_object(Bucket=bucket_name, Key=source_key)
    
        #print(f"Renamed {source_key} to {destination_key}")

